In [1]:
%autosave 60
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Autosaving every 60 seconds


# Computing the Conjunctive Normal Form in First Order Logic

In order to convert a formula $f$ from first order logic into a set of clauses we have to perform the following steps:
<ol>
<li>Eliminate biconditionals,</li>
<li>eliminate conditionals,</li>
<li>transform the formula into negation normal form,</li>
<li>transform the formula into prenex normal form,</li>
<li>skolemize the formula, and</li>
<li>transform the formula into clauses in set notation.</li>
</ol>

When converting formulas into conjunctive normal form, we assume that the formulas are *pure*, where we define a formula $f$ as *pure* if all quantifiers appearing in $f$ use **different** variables.  For example, the formula
$$ \forall x: p(x) \wedge \forall x: q(x)$$
is **not** *pure*, because there are two quantifiers that both bind the variable $x$.  We can rewrite this formulas as a *pure* formula as follows:
$$ \forall x: p(x) \wedge \forall y: q(y)$$

## Auxilliary Functions

Formulas are represented as nested tuples.  In order to convert a string into a nested tuple we use the <tt>LogicParser</tt> that is found in the module <tt>folParser</tt>.

In [2]:
import folParser as fp

The function $\texttt{parse}(s)$ takes a string $s$ which is a formula from frist order logic and turns this string into a nested tuple.

In [3]:
def parse(s):
    return fp.LogicParser(s).parse()

For testing purposes, the following formula is used.  This formula specifies the notion of a grandparent.

In [4]:
s  = '∀g:∀c:(Grandparent(g, c) ↔ ∃p: (Parent(g, p) ∧ Parent(p, c)))'
f1 = parse(s)
f1

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

The function $\texttt{apply}(t, σ)$ takes an object $t$ and a <em style="color:blue;">variable substitution</em> $\sigma$ which is of the form $\{x: s\}$ and replaces every occurrence of the variable $x$ in the object $t$ with the term $s$.  The object $t$ is either a term, a formula, a clause (represented as a <tt>frozenset</tt> of literals), or a set of clauses.

In [5]:
def apply(t, σ):
    "Apply the substitution σ to the term t."
    if isinstance(t, set):                           # t is a set of clauses
        return { apply(c, σ) for c in t }
    if isinstance(t, frozenset):                     # t is a clause
        return frozenset({ apply(l, σ) for l in t })
    if isinstance(t, str):                           # t is a variable
        if t in σ:
            return σ[t]
        else:
            return t
    else: 
        f  = t[0]
        ts = t[1:]
        return (f,) + tuple(apply(s, σ) for s in ts)

In [6]:
f1

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [7]:
apply(f1, { 'g': 'x', 'p': 'y', 'c': 'z' })

('∀',
 'x',
 ('∀',
  'z',
  ('↔',
   ('Grandparent', 'x', 'z'),
   ('∃', 'y', ('∧', ('Parent', 'x', 'y'), ('Parent', 'y', 'z'))))))

The function $\texttt{boundVariables}(f)$ computes the set of variables that are bound in the formula $f$.

In [8]:
def boundVariables(f):
    if f[0] in ('∀', '∃'):
        _, x, g = f
        return { x } | boundVariables(g)
    if f[0] == '⊤':
        return set()
    if f[0] == '⊥':
        return set()
    if f[0] == '¬':
        g  = f[1]
        return boundVariables(g)
    if f[0] in ('∧', '∨', '→', '↔'):
        _, g, h = f
        return boundVariables(g) | boundVariables(h)
    return set()  # f must be an atomic formula

In [9]:
f1

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [10]:
boundVariables(f1)

{'c', 'g', 'p'}

The function $\texttt{allVariables}(f)$ computes the set of all variables that occur in terms inside $f$. The object $f$ is either a formula or a term.

In [11]:
def allVariables(f):
    if isinstance(f, str):  # f is a variable
        return { f }
    if f[0] in ('∀', '∃'):
        _, _, g = f
        return allVariables(g)
    if f[0] == '⊤':
        return set()
    if f[0] == '⊥':
        return set()
    if f[0] == '¬':
        g  = f[1]
        return allVariables(g)
    if f[0] in ('∧', '∨', '→', '↔'):
        _, g, h = f
        return allVariables(g) | allVariables(h)
    args = f[1:]
    return { x for t in args for x in allVariables(t) }

In [12]:
f1

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [13]:
allVariables(f1)

{'c', 'g', 'p'}

Below we import the module <tt>string</tt> because it provides a definition of all lower case characters.

In [14]:
import string
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

The function $\texttt{renameBoundVariables}(f)$ takes a first order formula $f$ and replaces all bound variables by **new** variables.  The global variable <tt style="color:blue;">varCounter</tt> is used to generate new variable names.

In [15]:
def renameBoundVariables(f):
    BoundVs = boundVariables(f)
    NewVars = set(string.ascii_lowercase) - BoundVs - allVariables(f)
    NewVars = list(NewVars)
    sigma   = { x: NewVars[i] for (i, x) in enumerate(BoundVs) }
    return apply(f, sigma)

In [16]:
f1

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [17]:
renameBoundVariables(f1)

('∀',
 's',
 ('∀',
  'z',
  ('↔',
   ('Grandparent', 's', 'z'),
   ('∃', 'd', ('∧', ('Parent', 's', 'd'), ('Parent', 'd', 'z'))))))

## Elimination Biconditionals

The function $\texttt{eliminateBiconditional}(f)$ takes a formula $f$ from first order logic and eliminates all occurrences of the operator '↔' from this formula.  This is done by using the following equivalence:
$$ (f \leftrightarrow g) \;\Leftrightarrow\; (f \rightarrow g) \wedge (g \rightarrow f) $$
In order to ensure that the resulting formula is *pure*, we have to rename the bound variables in the formula $g \rightarrow f$.

In [18]:
def eliminateBiconditional(f):
    "Eliminate the logical operator '↔' from the formula f."
    if f[0] == '↔':
        g, h = f[1:]
        ge   = eliminateBiconditional(g)
        he   = eliminateBiconditional(h)
        return ('∧', ('→', ge, he), renameBoundVariables(('→', he, ge)))
    if f[0] == '⊤':
        return f
    if f[0] == '⊥':
        return f
    if f[0] == '¬':
        g  = f[1]
        ge = eliminateBiconditional(g)
        return ('¬', ge)
    if f[0] in ('∧', '∨', '→'):
        op, g, h = f
        ge       = eliminateBiconditional(g)
        he       = eliminateBiconditional(h)
        return (op, ge, he)
    if f[0] in ('∀', '∃'):
        q, x, g = f
        ge      = eliminateBiconditional(g)
        return (q, x, ge)
    return f              # f must be an atomic formula

In [19]:
f1

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [20]:
f2 = eliminateBiconditional(f1)
f2

('∀',
 'g',
 ('∀',
  'c',
  ('∧',
   ('→',
    ('Grandparent', 'g', 'c'),
    ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c')))),
   ('→',
    ('∃', 's', ('∧', ('Parent', 'g', 's'), ('Parent', 's', 'c'))),
    ('Grandparent', 'g', 'c')))))

## Eliminating Conditionals

The function $\texttt{eliminateConditional}(f)$ takes a formula $f$ from first order logic and eliminates all occurrences of the operator '→' from this formula.  This is done by using the following equivalence:
$$ (f \rightarrow g) \;\Leftrightarrow\; (\neg f \vee g) $$

In [21]:
def eliminateConditional(f):
    "Eliminate the logical operator '→' from f."
    if f[0] == '→':
        g, h = f[1:]
        ge   = eliminateConditional(g)
        he   = eliminateConditional(h)
        return ('∨', ('¬', ge), he)
    if f[0] == '⊤':
        return f
    if f[0] == '⊥':
        return f
    if f[0] == '¬':
        g  = f[1]
        ge = eliminateConditional(g)
        return ('¬', ge)
    if f[0] in ('∧', '∨'):
        op, g, h = f
        ge       = eliminateConditional(g)
        he       = eliminateConditional(h)
        return (op, ge, he)
    if f[0] in ('∀', '∃'):
        q, x, g = f
        ge      = eliminateConditional(g)
        return (q, x, ge)
    return f  # f must be an atomic formula

In [22]:
f2

('∀',
 'g',
 ('∀',
  'c',
  ('∧',
   ('→',
    ('Grandparent', 'g', 'c'),
    ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c')))),
   ('→',
    ('∃', 's', ('∧', ('Parent', 'g', 's'), ('Parent', 's', 'c'))),
    ('Grandparent', 'g', 'c')))))

In [23]:
f3 = eliminateConditional(f2)
f3

('∀',
 'g',
 ('∀',
  'c',
  ('∧',
   ('∨',
    ('¬', ('Grandparent', 'g', 'c')),
    ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c')))),
   ('∨',
    ('¬', ('∃', 's', ('∧', ('Parent', 'g', 's'), ('Parent', 's', 'c')))),
    ('Grandparent', 'g', 'c')))))

## Negation Normal Form

The function $\texttt{nnf}(f)$ computes the <em style="color:blue;">negation normal form</em> of $f$, while $\texttt{neg}(f)$ computes the *negation normal form* of $\neg f$.  The expression $\texttt{nnf}(f)$ is defined recursively as follows:
<ol>
    <li> $\texttt{nnf}(\neg \texttt{F}) = \texttt{neg}(\texttt{F})$, </li>
    <li> $\texttt{nnf}(\texttt{F}_1 \wedge \texttt{F}_2) = 
          \texttt{nnf}(\texttt{F}_1) \wedge \texttt{nnf}(\texttt{F}_2)$,</li>
    <li> $\texttt{nnf}(\texttt{F}_1 \vee \texttt{F}_2) = 
          \texttt{nnf}(\texttt{F}_1) \vee \texttt{nnf}(\texttt{F}_2)$.</li>
    <li> $\texttt{nnf}(\forall x: F) = \forall x: \texttt{nnf}(\texttt{F})$.</li>
    <li> $\texttt{nnf}(\exists x: F) = \exists x: \texttt{nnf}(\texttt{F})$.</li>
</ol>

The auxiliary function $\texttt{neg}$ is also defined recursively:
<ol>
    <li> $\texttt{neg}(p) = \texttt{nnf}(\neg p) = \neg p$ for all propositional variables $p$,</li>
    <li> $\texttt{neg}(\neg F) = \texttt{nnf}(\neg \neg F) = \texttt{nnf}(F)$,</li>
    <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(F_1 \wedge F_2 \bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg(F_1 \wedge F_2)\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1 \vee \neg F_2\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1\bigr) \vee \texttt{nnf}\bigl(\neg F_2\bigr) \\[0.1cm]
       = & \texttt{neg}(F_1) \vee \texttt{neg}(F_2).
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(F_1 \wedge F_2 \bigr) = \texttt{neg}(F_1) \vee \texttt{neg}(F_2)$.</li>
    <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(F_1 \vee F_2 \bigr)        \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg(F_1 \vee F_2) \bigr)  \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1 \wedge \neg F_2 \bigr)  \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1\bigr) \wedge \texttt{nnf}\bigl(\neg F_2 \bigr)  \\[0.1cm]
       = & \texttt{neg}(F_1) \wedge \texttt{neg}(F_2). 
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(F_1 \vee F_2 \bigr) = \texttt{neg}(F_1) \wedge \texttt{neg}(F_2)$.</li>
    <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(\forall x: F \bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg \forall x: F\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\exists x: \neg F\bigr) \\[0.1cm]
       = & \exists x: \texttt{nnf}(\neg F)           \\[0.1cm]
       = & \exists x: \texttt{neg}(F).
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(\forall x: F \bigr) = \exists x: \texttt{neg}(F)$.</li>
      <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(\exists x: F \bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg \exists x: F\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\forall x: \neg F\bigr) \\[0.1cm]
       = & \forall x: \texttt{nnf}(\neg F)           \\[0.1cm]
       = & \forall x: \texttt{neg}(F).
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(\exists x: F \bigr) = \forall x: \texttt{neg}(F)$.</li>
</ol>

In [24]:
def nnf(f):
    "Compute the negation normal form of f."
    if f[0] == '⊤':
        return f
    if f[0] == '⊥':
        return f
    if f[0] == '¬':
        g = f[1]
        return neg(g)
    if f[0] in ('∧', '∨'):
        op, g, h = f
        return (op, nnf(g), nnf(h))
    if f[0] in ('∀', '∃'):
        q, x, g = f
        return (q, x, nnf(g))
    return f                     # f must be atomic here

def neg(f):
    "Compute the negation normal form of ¬f."
    if f[0] == '⊤':
        return ('⊥',)
    if f[0] == '⊥':
        return ('⊤',)
    if f[0] == '¬':
        g = f[1]
        return nnf(g)
    if f[0] == '∧':
        g, h = f[1:]
        return ('∨', neg(g), neg(h))
    if f[0] == '∨':
        g, h = f[1:]
        return ('∧', neg(g), neg(h))
    if f[0] == '∀':
        q, x, g = f
        return ('∃', x, neg(g))
    if f[0] == '∃':
        q, x, g = f
        return ('∀', x, neg(g))
    return ('¬', f)              # f must be atomic here

In [25]:
f3

('∀',
 'g',
 ('∀',
  'c',
  ('∧',
   ('∨',
    ('¬', ('Grandparent', 'g', 'c')),
    ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c')))),
   ('∨',
    ('¬', ('∃', 's', ('∧', ('Parent', 'g', 's'), ('Parent', 's', 'c')))),
    ('Grandparent', 'g', 'c')))))

In [26]:
f4 = nnf(f3)
f4

('∀',
 'g',
 ('∀',
  'c',
  ('∧',
   ('∨',
    ('¬', ('Grandparent', 'g', 'c')),
    ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c')))),
   ('∨',
    ('∀',
     's',
     ('∨', ('¬', ('Parent', 'g', 's')), ('¬', ('Parent', 's', 'c')))),
    ('Grandparent', 'g', 'c')))))

## Prenex Normal Form

In the following we assume that all quantifiers that occur in a formula bind **different** variables, i.e. we assume that the formulas are **pure**.  If this assumption is not satisfied, then the functions given below produce garbage.

A <em style="color:blue;">quantifier tuple</em> is a tuple of the following form:
$$ (Q_1, x_1, \cdots, Q_n, x_n) $$
Here, the $Q_i$ denote quantifiers, i.e. we have $Q_i \in \{\forall, \exists\}$, while the $x_i$ are variables.  The function $\texttt{mergeQuantifiers}(T_1, T_2)$ takes two quantifier tuples $T_1$ and $T_2$ as arguments and merges them into a new quantifier tuple such that the relative order of the quantifiers remains the same, i.e. if both $Q_1, x_1$ and $Q_2, x_2$ occur in $T_1$ and $Q_1, x_1$ occurs before $Q_2, x_2$, then $Q_1, x_1$ will occur before $Q_2, x_2$ in the result.

In [27]:
def mergeQuantifiers(Q1, Q2):
    if Q1 == ():
        return Q2
    if Q2 == ():
        return Q1
    if Q1[0] == '∃':  # extract existential quantifiers first
        return Q1[:2] + mergeQuantifiers(Q1[2:], Q2)
    if Q2[0] == '∃':
        return Q2[:2] + mergeQuantifiers(Q1, Q2[2:])
    return Q1[:2] + mergeQuantifiers(Q1[2:], Q2)

In [28]:
mergeQuantifiers(('∀', 'x', '∃', 'y', '∀', 'z'), ('∃', 'u', '∀', 'v', '∃', 'w'))

('∃', 'u', '∀', 'x', '∃', 'y', '∀', 'z', '∀', 'v', '∃', 'w')

Given a formula $f$, the function $\texttt{extractQuantifiers}(f)$ returns a pairs $(T, m)$, where $T$ is a quantifier tuple and $m$ is the <em style="color:blue;">matrix</em> of the formula $f$, where the matrix of a formula is defined as the part that remains when all quantifiers have been extracted.

In [29]:
def extractQuantifiers(f):
    if f[0] in ('⊤', '⊥'):
        return (), f
    if f[0] == '¬':
        return (), f
    if f[0] in ('∧', '∨'):
        op, g, h = f
        qg, gm = extractQuantifiers(g)
        qh, hm = extractQuantifiers(h)
        # this works because f is pure
        return mergeQuantifiers(qg, qh), (op, gm, hm)
    if f[0] in ('∀', '∃'):
        q, x, g = f
        qg, gm  = extractQuantifiers(g)
        return (q, x) + qg, gm
    return (), f             # f must be atomic here

In [30]:
f4

('∀',
 'g',
 ('∀',
  'c',
  ('∧',
   ('∨',
    ('¬', ('Grandparent', 'g', 'c')),
    ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c')))),
   ('∨',
    ('∀',
     's',
     ('∨', ('¬', ('Parent', 'g', 's')), ('¬', ('Parent', 's', 'c')))),
    ('Grandparent', 'g', 'c')))))

In [31]:
Qs, f5 = extractQuantifiers(f4)
Qs, f5

(('∀', 'g', '∀', 'c', '∃', 'p', '∀', 's'),
 ('∧',
  ('∨',
   ('¬', ('Grandparent', 'g', 'c')),
   ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))),
  ('∨',
   ('∨', ('¬', ('Parent', 'g', 's')), ('¬', ('Parent', 's', 'c'))),
   ('Grandparent', 'g', 'c'))))

Given a qantifier tuple $Qs$ and a matrix $m$, the call $\texttt{attachQuantifiers}(Qs, m)$ combines the quantifiers and the matrix into a quantified formula.

In [32]:
def attachQuantifiers(Qs, m):
    if Qs == ():
        return m
    (Q, x) = Qs[:2]
    Qr     = Qs[2:]
    return (Q, x, attachQuantifiers(Qr, m))

In [33]:
f5

('∧',
 ('∨',
  ('¬', ('Grandparent', 'g', 'c')),
  ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))),
 ('∨',
  ('∨', ('¬', ('Parent', 'g', 's')), ('¬', ('Parent', 's', 'c'))),
  ('Grandparent', 'g', 'c')))

In [34]:
f6 = attachQuantifiers(Qs, f5)
f6

('∀',
 'g',
 ('∀',
  'c',
  ('∃',
   'p',
   ('∀',
    's',
    ('∧',
     ('∨',
      ('¬', ('Grandparent', 'g', 'c')),
      ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))),
     ('∨',
      ('∨', ('¬', ('Parent', 'g', 's')), ('¬', ('Parent', 's', 'c'))),
      ('Grandparent', 'g', 'c')))))))

The variable $\texttt{skolemCounter}$ is a global variable that is needed to create unique Skolem constants.  

In [35]:
skolemCounter = 0

In [36]:
def skolemConstant():
    global skolemCounter
    skolemCounter += 1
    return 'sk' + str(skolemCounter)

The function $\texttt{skolemize}(f, \texttt{Vs})$ takes a formula $f$ and a tuple of variables $\texttt{Vs}$ and *skolemizes* the formula $f$, i.e. it replaces all existentially quantified variables by appropriate *Skolem functions*.  For skolemization to work correctly, <b style="color:red;">we have to assume that $f$ does not contain free variables</b>!

In [37]:
def skolemize(f, Vs):
    if f[0] == '∃':
        x, g = f[1:]
        t = (skolemConstant(), Vs) 
        σ = { x: t }
        return skolemize(apply(g, σ), Vs)
    if f[0] == '∀':
        x, g = f[1:]
        return skolemize(g, Vs + (x,))
    return f                            # at this point f must be an atomic formula    

In [38]:
f6

('∀',
 'g',
 ('∀',
  'c',
  ('∃',
   'p',
   ('∀',
    's',
    ('∧',
     ('∨',
      ('¬', ('Grandparent', 'g', 'c')),
      ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))),
     ('∨',
      ('∨', ('¬', ('Parent', 'g', 's')), ('¬', ('Parent', 's', 'c'))),
      ('Grandparent', 'g', 'c')))))))

In [39]:
f7 = skolemize(f6, ())
f7

('∧',
 ('∨',
  ('¬', ('Grandparent', 'g', 'c')),
  ('∧',
   ('Parent', 'g', ('sk1', ('g', 'c'))),
   ('Parent', ('sk1', ('g', 'c')), 'c'))),
 ('∨',
  ('∨', ('¬', ('Parent', 'g', 's')), ('¬', ('Parent', 's', 'c'))),
  ('Grandparent', 'g', 'c')))

## Conversion to Clauses

The function $\texttt{cnf}(f)$ takes a *skolemized* formula $f$ from first order logic that is in *negation normal form* and returns the *conjunctive normal form* of $f$ in *set notation*.

In [40]:
def cnf(f):
    if f[0] == '⊤':
        return set()
    if f[0] == '⊥':
        return {frozenset()}
    if f[0] == '¬':
        return { frozenset({f}) }
    if f[0] == '∧':
        g, h = f[1:]
        return cnf(g) | cnf(h)
    if f[0] == '∨':
        g, h = f[1:]
        return { k1 | k2 for k1 in cnf(g) for k2 in cnf(h) }
    return { frozenset({f}) }    # f is atomic

In [41]:
f8 = cnf(f7)
f8

{frozenset({('Parent', 'g', ('sk1', ('g', 'c'))),
            ('¬', ('Grandparent', 'g', 'c'))}),
 frozenset({('Grandparent', 'g', 'c'),
            ('¬', ('Parent', 'g', 's')),
            ('¬', ('Parent', 's', 'c'))}),
 frozenset({('Parent', ('sk1', ('g', 'c')), 'c'),
            ('¬', ('Grandparent', 'g', 'c'))})}

## Putting Everything Together

The function $f$ takes a **pure** formula $f$ from first order logic and transforms $f$ into a set of first order clauses.  Furthermore, $f$ **must not** contain free variables.

In [42]:
def normalize(f):
    f1     = eliminateBiconditional(f)
    f2     = eliminateConditional(f1)
    f3     = nnf(f2)
    Qs, f4 = extractQuantifiers(f3)
    f5     = attachQuantifiers(Qs, f4)
    f6     = skolemize(f5, ())
    f7     = cnf(f6)
    return f7

In [43]:
def test(s):
    f = fp.LogicParser(s).parse()
    print(f'The knf of {s} is:')
    print(prettify(normalize(f)))

In [44]:
def prettify(M):
    """Turn the set of frozen sets M into a string that looks like a set of sets.
       M is assumed to be the power set of some set.
    """
    if M == set():
        return '{}'
    result = "{\n"
    for A in M:
        if A == frozenset(): 
            result += "{},\n"
        else:
            result += "    " + str(set(A)) + ",\n" # A is converted from a frozen set to a set
    result = result[:-2] # remove the trailing substring ", "
    result += "\n}"
    return result

In [45]:
test(s)

The knf of ∀g:∀c:(Grandparent(g, c) ↔ ∃p: (Parent(g, p) ∧ Parent(p, c))) is:
{
    {('Parent', 'g', ('sk2', ('g', 'c'))), ('¬', ('Grandparent', 'g', 'c'))},
    {('¬', ('Parent', 's', 'c')), ('¬', ('Parent', 'g', 's')), ('Grandparent', 'g', 'c')},
    {('Parent', ('sk2', ('g', 'c')), 'c'), ('¬', ('Grandparent', 'g', 'c'))}
}
